# 소행성 관측 계획 수립하기

* 이 노트북을 구글 코랩에서 실행하고자 한다면 [파일] - [드라이브에 사본 저장]을 하여 본인의 소유로 만든 후에 코드를 실행하거나 수정할 수 있습니다.

* 이 파일은 실제 수업에 사용하므로 필요에 따라 예고 없이 변경될 수 있습니다.

* If you have any questions or comments on this document, please email me(Kiehyun.Park@gmail.com).

* 이 파일(문서)는 공교육 현장에서 수업시간에 자유롭게 사용할 수 있으나, 다른 목적으로 사용할 시에는 사전에 연락을 주셔서 상의해 주시기 바랍니다.

이 자료는 What's observable로 부터 특정한 날 관측할 수 있는 소행성의 목록을 만드는 과정을 설명합니다.

## 필요한 환경



### 한글 폰트 설치 (matplotlib에서 한글 폰트 깨질 경우)

구글 코랩에서 현재의 Jupyter notebook을 실행한다면 matplotlib 그래프에 한글을 사용하기 위해서 폰트를 설치해야 합니다.

아래 코드의 주석을 제거한 후에 셀을 실해 주세요. 그리고 나서 런타임 다시 시작을 해 줘야 matplotlib 그래프에서 한글을 사용할 수 있습니다.

In [24]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

#### 런타임 다시 시작

위의 셀을 실행한 다음 반드시 다음 과정을 수행해야 합니다.

* [메뉴]-[런타임]-[런터임 다시 시작]

* [메뉴]-[런타임]-[이전 셀 실행]

#### 한글 폰트 사용

위에서 한글 폰트를 설치하고, 런타임 다시시작을 했다면 구글 코랩에서 폰트 경로를 설정하여 한글 사용이 가능해 집니다.

In [25]:
#visualization
import matplotlib as mpl
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록
%matplotlib inline

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정
# plt.rc('font', family='NanumBarunGothic')

# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

plt.rcParams.update({'figure.max_open_warning': 0})

import warnings
warnings.filterwarnings('ignore')

### 모듈 설치 및 버전 확인

아래 셀을 실행하면 이 노트북을 실행하는데 필요한 모듈을 설치하고 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [26]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, astroquery, requests, astropy, certifi, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"**** {pkg} module is now installed.")
    else:
        print(f"******** {pkg} module is already installed.")
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

******** numpy module is already installed.
******** pandas module is already installed.
******** matplotlib module is already installed.
******** astroquery module is already installed.
******** requests module is already installed.
******** astropy module is already installed.
******** certifi module is already installed.
******** version_information module is already installed.
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2024-09-17 12:20:31 (UTC = GMT+0000) 
0 Python     3.10.12 64bit [GCC 11.4.0]
1 IPython    7.34.0
2 OS         Linux 6.1.85+ x86_64 with glibc2.35
3 numpy      1.26.4
4 pandas     2.1.4
5 matplotlib 3.7.1
6 astroquery 0.4.7
7 requests   2.32.3
8 astropy    6.1.3
9 certifi    2024.08.30
10 version_information 1.0.4


## 날짜와 시간 다루기

### datetime.datetime

Python 내장 모듈인 datetime을 사용하면 날짜와 시각 객체를 다룰 수 있습니다.

#### 표준시와 세계시

In [27]:
from datetime import datetime, timedelta, timezone

dt_kst_now = datetime.now()
print("현재 시각(KST=GMT+9) :", dt_kst_now)

timezone_utc = timezone(timedelta(hours=-9))
dt_utc_now = dt_kst_now.astimezone(timezone_utc)
print("현재 시각(UTC) :", dt_utc_now)

현재 시각(KST=GMT+9) : 2024-09-17 12:20:31.735661
현재 시각(UTC) : 2024-09-17 03:20:31.735661-09:00


#### fits header의 시간 형식

fits header의 시간 형식은 '2023-09-11T20:19:18'의 형태를 띠고 있는데 datetime.strptime을 이용하면 날짜시간 형식을 바로 datetime 객체로 만들 수 있습니다.

이를 세계시로 바꾸려면 timedelta를 이용할 수 있습니다.

In [28]:
start_dt_kst_str = '2023-09-11T20:19:18'
end_dt_kst_str = '2023-12-31T00:00:00'

start_dt_kst = datetime.strptime(start_dt_kst_str, '%Y-%m-%dT%H:%M:%S')
print(start_dt_kst)
print(type(start_dt_kst))

start_dt_utc = start_dt_kst + timedelta(hours=-9)
print(start_dt_utc)
print(type(start_dt_utc))

2023-09-11 20:19:18
<class 'datetime.datetime'>
2023-09-11 11:19:18
<class 'datetime.datetime'>


#### string 형태로 변환하기

datetime 객체를 string으로 바꿀때는 strftime 메쏘들ㄹ 이용하면 됩니다.

In [29]:

start_dt_utc_str = start_dt_utc.strftime('%Y-%m-%dT%H:%M:%S')
print("start_dt_utc_str :", start_dt_utc_str)
print("type(start_dt_utc_str) :", type(start_dt_utc_str))


start_dt_utc_str : 2023-09-11T11:19:18
type(start_dt_utc_str) : <class 'str'>


#### datetime 이용하여 list 생성

아래와 같이 하면, 시작 시각과 끝나는 시각, 그리고 시간 간격을 정해 주고, Python의 datatime 객체를 이용하여 일정 시간 간격의 목록(list)을 만을 수 있다.

In [30]:
from datetime import datetime, timedelta
start_time_text = "2017-01-01"
end_time_text = "2017-06-01"
dt = 1
def datetime_range(start_dt, end_dt, delta):
    '''
        Parameters
        ----------
        start_dt : datetime.datetime
        end_dt : datetime.datetime
        delta : int
    '''
    current = start_dt
    while current < end_dt:
        yield current
        current += delta

dts = [dt.strftime('%Y-%m-%d') for dt in
            datetime_range(datetime.fromisoformat(start_time_text), datetime.fromisoformat(end_time_text),
            timedelta(days=dt))]

print("len(dts):", len(dts))
print("dts", dts)

len(dts): 151
dts ['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08', '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12', '2017-01-13', '2017-01-14', '2017-01-15', '2017-01-16', '2017-01-17', '2017-01-18', '2017-01-19', '2017-01-20', '2017-01-21', '2017-01-22', '2017-01-23', '2017-01-24', '2017-01-25', '2017-01-26', '2017-01-27', '2017-01-28', '2017-01-29', '2017-01-30', '2017-01-31', '2017-02-01', '2017-02-02', '2017-02-03', '2017-02-04', '2017-02-05', '2017-02-06', '2017-02-07', '2017-02-08', '2017-02-09', '2017-02-10', '2017-02-11', '2017-02-12', '2017-02-13', '2017-02-14', '2017-02-15', '2017-02-16', '2017-02-17', '2017-02-18', '2017-02-19', '2017-02-20', '2017-02-21', '2017-02-22', '2017-02-23', '2017-02-24', '2017-02-25', '2017-02-26', '2017-02-27', '2017-02-28', '2017-03-01', '2017-03-02', '2017-03-03', '2017-03-04', '2017-03-05', '2017-03-06', '2017-03-07', '2017-03-08', '2017-03-09', '2017-03-10', '2017-03-11', '

### astropy.time

표준시나 세계시 정도는 datetime을 사용해도 무방하지만 천문학에서 사용하는 시간에는 조금 부족합니다. 천문학에서 사용하는 시간계는 datetime 모듈보다는 astropy.time 모듈을 사용하기를 권장합니다.

#### 세계시

In [31]:
from astropy.time import Time

t_kst_now = Time.now()
print("현재 시각(KST=GMT+9) :", t_kst_now)

t_ut_now = Time(datetime.utcnow(), scale='utc', location=('127d', '37d'))
print("현재 시각(UTC) :", t_ut_now)

print("type(t_ut_now) ;", type(t_ut_now))

현재 시각(KST=GMT+9) : 2024-09-17 12:20:31.906452
현재 시각(UTC) : 2024-09-17 12:20:31.951186
type(t_ut_now) ; <class 'astropy.time.core.Time'>


#### dir함수로 메쏘드 알아보기


In [32]:
print("dir(t_ut_now) ;", dir(t_ut_now))

dir(t_ut_now) ; ['FORMATS', 'SCALES', 'T', '_APPLICABLE_FUNCTIONS', '_METHOD_FUNCTIONS', '__abstractmethods__', '__add__', '__array_function__', '__array_priority__', '__bool__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_abc_impl', '_advanced_index', '_apply', '_astropy_column_attrs', '_call_erfa', '_format', '_get_delta_tdb_tt', '_get_delta_ut1_utc', '_get_time_fmt', '_id_cache', '_init_from_vals', '_location', '_make_value_equivalent', '_match_shape', '_ptp_decorator', '_ptp_impl', '_set_delta_tdb_tt', '_set_delta_ut1_utc', '_set_scale

#### Julian day 구하기

In [33]:
print("t_ut_now.to_value(format='jd') ;", t_ut_now.to_value(format='jd'))

t_ut_now.to_value(format='jd') ; 2460571.0142586944


#### 항성시

In [34]:
print("t_ut_now.sidereal_time('mean') :", t_ut_now.sidereal_time('mean'))

t_ut_now.sidereal_time('mean') : 20h36m14.68431983s


#### astropy.time 이용하여 list 생성

일정 시간 간격으로 시뮬레이션 하려면 시간 간격의 리스트를 미리 만들어 두면 편리할 것입니다. 다음은 Python의 astropy.time을 이용하여 일정 시간 간격의 목록(list)을 만드는 방법입니다.

In [35]:
import numpy as np
from astropy.time import Time, TimeDelta

start_time_text = "2023-10-06"
end_time_text = "2023-10-15"
dt = 1

def astropyTime_range(start_t, end_t, delta):
    '''
        Parameters
        ----------
        start_t : datetime.datetime
        end_t : datetime.datetime
        delta : int
    '''
    current = start_t
    while current < end_t:
        yield current
        current += delta

dts = [dt for dt in
        astropyTime_range(Time(start_time_text), Time(end_time_text),
        TimeDelta(dt, format='jd'))]

print(len(dts))
print(dts)

9
[<Time object: scale='utc' format='iso' value=2023-10-06 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-07 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-08 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-09 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-10 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-11 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-12 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-13 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-14 00:00:00.000>]


## What's observable

[What's observable](https://ssd.jpl.nasa.gov/tools/sbwobs.html) 에서는 관측소 코드와 관측 조건들을 설정하여 소행성과 혜성의 관측 대상의 정보를 얻을 수 있습니다.


### Horizons API

[Horizons API](https://ssd-api.jpl.nasa.gov/doc/horizons.html)를 제공하고 있어 requests로 받아아올 url을 만들수 있습니다.



#### 검색 조건 설정

먼저 조건들을 변수로 설정해 보겠습니다.

In [36]:
#############################################
# variables
mpc_code='P64' # Observer Location: GSHS Observatory, Suwon [code: P64]

#관측 시각 설정 (현재 시각)
t_ut_now = Time(datetime.now(), scale='utc', location=('127d', '37d'))
print("type(obs_dt_utc) :", type(t_ut_now))
print("현재 시각(UTC) :", t_ut_now)

#관측 시각 설정 (임의의 시각)
# t_ut_now = Time(datetime(2023, 10, 2, 10, 0, 0), scale='utc', location=('127d', '37d'))
# print("type(obs_dt_utc) :", type(t_ut_now))
# print("현재 시각(UTC) :", t_ut_now)

obs_datetime = t_ut_now.strftime('%Y-%m-%dT%H:%M:%S')
print("obs_datetime :", obs_datetime)
obs_date = t_ut_now.strftime('%Y%m%d')
print("obs_date :", obs_date)

elev_min=60  #
time_min=15
vmag_min=6
vmag_max=13
list_num=100
sort='trans'

type(obs_dt_utc) : <class 'astropy.time.core.Time'>
현재 시각(UTC) : 2024-09-17 12:20:32.485166
obs_datetime : 2024-09-17T12:20:32
obs_date : 20240917


#### url 만들기

In [37]:
# Define API URL and SPK filename:
url = 'https://ssd-api.jpl.nasa.gov/sbwobs.api'
spk_filename = 'spk_file.bsp'

# Get the requested SPK-ID from the command-line:
if (len(sys.argv)) == 1:
  print("please specify SPK-ID on the command-line")
  sys.exit(2)
spkid = sys.argv[1]

# Build the appropriate URL for this API request:
# IMPORTANT: You must encode the "=" as "%3D" and the ";" as "%3B" in the
#            Horizons COMMAND parameter specification.
url += f"?sb-kind=a&mpc-code={mpc_code}&obs-time={obs_datetime}&elev-min={elev_min}&time-min={time_min}"
url += f"&vmag-max={vmag_max}&vmag-min={vmag_min}&optical=true&fmt-ra-dec=true&mag-required=true&output-sort={sort}&maxoutput={list_num}"

print("url :", url)

url : https://ssd-api.jpl.nasa.gov/sbwobs.api?sb-kind=a&mpc-code=P64&obs-time=2024-09-17T12:20:32&elev-min=60&time-min=15&vmag-max=13&vmag-min=6&optical=true&fmt-ra-dec=true&mag-required=true&output-sort=trans&maxoutput=100


### reauests.get

api에서 제공하는 내용을 json 형태로 받아 보겠습니다.

In [38]:
import json
import requests
# Submit the API request and decode the JSON-response:
response = requests.get(url)
try:
  data = json.loads(response.text)
  print(data)
except ValueError:
  print("Unable to decode JSON results")

{'signature': {'source': 'NASA/JPL Small-Body Observability API', 'version': '1.0'}, 'obs_constraints': {'fmt-ra-dec': 'true', 'vmag-min': '6', 'obs-time': '2024-09-17 12:20:32', 'elev-min': '60', 'mpc-code': 'P64', 'time-min': '15', 'optical': 'true', 'vmag-max': '13', 'output-sort': 'trans', 'maxoutput': '100', 'mag-required': 'true'}, 'sb_constraints': {'sb-kind': 'a'}, 'location': 'GSHS Observatory, Suwon', 'obs_night': {'sun_set_az': '273.3', 'sun_set': '2024-09-17 09:35', 'end_civil': '2024-09-17 10:01', 'end_nautical': '2024-09-17 10:32', 'end_astronomical': '2024-09-17 11:03', 'begin_astronomical': '2024-09-17 19:46', 'begin_nautical': '2024-09-17 20:18', 'begin_civil': '2024-09-17 20:48', 'sun_rise_az': '86.9', 'sun_rise': '2024-09-17 21:14', 'transit_phase': '1.00', 'transit': '2024-09-17 15:06 ', 'moon_set_phase': '1.00', 'moon_set': '2024-09-17 21:00 ', 'moon_rise_phase': '0.99', 'moon_rise': '2024-09-17 09:15 ', 'begin_dark': '2024-09-17 11:03', 'mid_dark': '2024-09-17 15:

안에 들어 있는 데이터를 확인해 보곘습니다.

In [39]:
print("type(data) :", type(data))
print("data.keys :", data.keys)

type(data) : <class 'dict'>
data.keys : <built-in method keys of dict object at 0x7d35a47d9180>


 아마도 data['fields']에 있는 것이 필요한 자료인것 같습니다.

In [40]:
print(data['obs_constraints'])
print(data['sb_constraints'])
print(data['fields'])
print(data['data'])
print(type(data['fields']))
print(len(data['fields']))
print(len(data['data'][0]))


{'fmt-ra-dec': 'true', 'vmag-min': '6', 'obs-time': '2024-09-17 12:20:32', 'elev-min': '60', 'mpc-code': 'P64', 'time-min': '15', 'optical': 'true', 'vmag-max': '13', 'output-sort': 'trans', 'maxoutput': '100', 'mag-required': 'true'}
{'sb-kind': 'a'}
['Designation', 'Full name', 'Rise time', 'Transit time', 'Set time', 'Max. time observable', 'R.A.', 'Dec.', 'Vmag', 'Helio. range (au)', 'Topo.range (au)', 'Object-Observer-Sun (deg)', 'Object-Observer-Moon (deg)', 'Galactic latitude (deg)']
[['1036', '1036 Ganymed (A924 UB)', '09:43*', '12:14', '14:44', '03:41', '20:29:57', '+51 27\'42"', '9.48', '1.25', '0.442', '112.94', '68.64', '7.26'], ['176', '176 Iduna (A877 TB)', '12:16', '13:10', '14:03', '01:47', '21:26:01', '+09 53\'25"', '12.9', '2.81', '1.93', '144.04', '32.78', '-27.81'], ['137', '137 Meliboea (A874 HA)', '14:53', '15:20', '15:47', '00:53', '23:36:39', '+07 56\'30"', '11.8', '2.60', '1.61', '170.06', '15.41', '-50.51'], ['109', '109 Felicitas (A869 TA)', '16:14', '16:52',

### dataframe 만들기

아무래도 가장 다루기 쉬운 데이타 형태인 dataframe으로 만들어 보겠습니다.

In [41]:
import pandas as pd

df = pd.DataFrame.from_dict(data['data'])
df = df.set_axis((data['fields']), axis=1)
df['OBSdate(UT)'] = obs_date
df

,Designation,Full name,Rise time,Transit time,Set time,Max. time observable,R.A.,Dec.,Vmag,Helio. range (au),Topo.range (au),Object-Observer-Sun (deg),Object-Observer-Moon (deg),Galactic latitude (deg),OBSdate(UT)
0,1036,1036 Ganymed (A924 UB),09:43*,12:14,14:44,03:41,20:29:57,"+51 27'42""",9.48,1.25,0.442,112.94,68.64,7.26,20240917
1,176,176 Iduna (A877 TB),12:16,13:10,14:03,01:47,21:26:01,"+09 53'25""",12.9,2.81,1.93,144.04,32.78,-27.81,20240917
2,137,137 Meliboea (A874 HA),14:53,15:20,15:47,00:53,23:36:39,"+07 56'30""",11.8,2.60,1.61,170.06,15.41,-50.51,20240917
3,109,109 Felicitas (A869 TA),16:14,16:52,17:29,01:15,01:08:29,"+08 34'00""",11.6,2.07,1.12,156.16,31.29,-54.24,20240917
4,547,547 Praxedis (A904 TE),16:14,17:14,18:15,02:01,01:31:30,"+10 37'23""",12.9,2.14,1.21,150.14,37.28,-51.22,20240917
5,203,203 Pompeja (A879 SA),16:06,17:23,18:40,02:34,01:39:43,"+12 47'00""",12.9,2.60,1.69,147.42,40.05,-48.62,20240917
6,19,19 Fortuna (A852 QA),16:20,17:34,18:47,02:27,01:50:38,"+12 16'20""",10.4,2.07,1.17,145.18,42.22,-48.29,20240917
7,10,10 Hygiea (A849 GA),16:03,17:46,19:30,03:27,02:03:37,"+17 51'02""",11.1,3.42,2.59,140.04,47.50,-41.97,20240917
8,270,270 Anahita (A887 TA),16:53,18:47,20:42*,02:53,03:04:41,"+20 27'23""",11.9,1.96,1.19,125.70,61.74,-32.79,20240917
9,33,33 Polyhymnia (A854 UB),17:14,19:05,20:55*,02:32,03:22:02,"+19 30'01""",11.6,2.05,1.33,122.07,65.29,-31.08,20240917


### 관측 계획 저장 폴더 생성

FITS 파일을 저장할 폴더를 "OBSplan_asteroid" 이라는 이름으로 생성해보겠습니다.

* 만약 리눅스 시스템 이라면 shell 명령어로 가능한데, "!"를 붙이면 shell 명령어를 실행할 수 있습니다.
> !mkdir OBSplan_asteroid

OS의 영향을 받지 않기 위하여 pathlib을 사용하여 다음과 같이 폴더를 생성합니다.

In [42]:
import os
from pathlib import Path
BASEPATH = Path("./")
save_dir_name = "OBSplan_asteroid"
print(f"BASEPATH: {BASEPATH}")

if not (BASEPATH/save_dir_name).exists():
    os.mkdir(str(BASEPATH/save_dir_name))
    print (f"{str(BASEPATH/save_dir_name)} is created...")
else :
    print (f"{str(BASEPATH/save_dir_name)} is already exist...")

BASEPATH: .
OBSplan_asteroid is already exist...


### dataframe 저장

In [43]:
df.to_csv(str(BASEPATH/save_dir_name/ f'asteroid_{obs_date}.csv'))

### (과제)

12월 1달 동안의 NEA 목록을 csv  파일로 저장하는 코드를 완성하세요.



In [ ]:
#(과제) 아래 코딩을 완성하여 제출하시오.
#############################################
# variables
mpc_code='P64' # Observer Location: GSHS Observatory, Suwon [code: P64]

elev_min=60 #
time_min=15
vmag_min=6
vmag_max=13
list_num=50
sort='trans'
#############################################

start_dt    = datetime(2024, 12, 1)
end_dt      = datetime(2025, 1, 1)
dt = 1

dts = [dt for dt in
            datetime_range(start_dt, end_dt,
            timedelta(days=dt))]

print("len(dts):", len(dts))
print("dts", dts)

df_all = pd.DataFrame()

for dt in dts :
    obs_datetime = dt.strftime('%Y-%m-%d %H:%M:%S')
    print("obs_datetime :", obs_datetime)
    obs_date = dt.strftime('%Y%m%d')
    print("obs_date :", obs_date)

    # Define API URL and SPK filename:
    url = 'https://ssd-api.jpl.nasa.gov/sbwobs.api'
    spk_filename = 'spk_file.bsp'

    # Get the requested SPK-ID from the command-line:
    if (len(sys.argv)) == 1:
        print("please specify SPK-ID on the command-line")
        sys.exit(2)
    spkid = sys.argv[1]

    # Build the appropriate URL for this API request:
    # IMPORTANT: You must encode the "=" as "%3D" and the ";" as "%3B" in the
    #            Horizons COMMAND parameter specification.
    url += f"?sb-kind=a&mpc-code={mpc_code}&obs-time={str(obs_datetime)}&elev-min={elev_min}&time-min={time_min}"
    url += f"&vmag-max={vmag_max}&vmag-min={vmag_min}&optical=true&fmt-ra-dec=true&mag-required=true&output-sort={sort}&maxoutput={list_num}"

    print("url :", url)

    # Submit the API request and decode the JSON-response:
    response = requests.get(url)
    try:
        data = json.loads(response.text)
        print(data)
    except ValueError:
        print("Unable to decode JSON results")

    df = pd.DataFrame.from_dict(data['data'])
    df = df.set_axis((data['fields']), axis=1)
    df['OBSdate(UT)'] = obs_date
    #print(df)

    df.to_csv(str(BASEPATH/save_dir_name/f"asteroid_{dt.strftime('%Y%m%d')}.csv"))

    df_all = pd.concat([df_all, df], axis = 0)

df_all.reset_index(inplace=True)
print("df_all :", df_all)

df_all.to_csv(str(BASEPATH/save_dir_name/f"asteroid_{dts[0].strftime('%Y%m%d')}-{dts[-1].strftime('%Y%m%d')}.csv"))

len(dts): 31
dts [datetime.datetime(2024, 12, 1, 0, 0), datetime.datetime(2024, 12, 2, 0, 0), datetime.datetime(2024, 12, 3, 0, 0), datetime.datetime(2024, 12, 4, 0, 0), datetime.datetime(2024, 12, 5, 0, 0), datetime.datetime(2024, 12, 6, 0, 0), datetime.datetime(2024, 12, 7, 0, 0), datetime.datetime(2024, 12, 8, 0, 0), datetime.datetime(2024, 12, 9, 0, 0), datetime.datetime(2024, 12, 10, 0, 0), datetime.datetime(2024, 12, 11, 0, 0), datetime.datetime(2024, 12, 12, 0, 0), datetime.datetime(2024, 12, 13, 0, 0), datetime.datetime(2024, 12, 14, 0, 0), datetime.datetime(2024, 12, 15, 0, 0), datetime.datetime(2024, 12, 16, 0, 0), datetime.datetime(2024, 12, 17, 0, 0), datetime.datetime(2024, 12, 18, 0, 0), datetime.datetime(2024, 12, 19, 0, 0), datetime.datetime(2024, 12, 20, 0, 0), datetime.datetime(2024, 12, 21, 0, 0), datetime.datetime(2024, 12, 22, 0, 0), datetime.datetime(2024, 12, 23, 0, 0), datetime.datetime(2024, 12, 24, 0, 0), datetime.datetime(2024, 12, 25, 0, 0), datetime.datetim

## 소행성 좌표 얻기

### 소행성 id 잘라내기

앞서 얻은 dataframe로 부터 소행성의 이름을 알아보자.

In [ ]:
#print("df :", df)
#print("df['Full name'] :", df['Full name'])
print("df['Full name'][0] :", df['Full name'][0])

#import re
#asteroid_names = re.findall('\(([^)]+)', df['Full name'][0])   #extracts string in bracket()
#print (asteroid_names[0])

asteroid_id = df['Full name'][0].split(" ")[0]   #extracts string in bracket()
print("asteroid_id :", asteroid_id)
obs_date = df['OBSdate(UT)'][0]

### astroquery.mpc

[astroquery.mpc](https://astroquery.readthedocs.io/en/latest/mpc/mpc.html) 를 이용하여 시간에 따른 소행성의 좌표를 얻어보자.

In [ ]:
from astroquery.mpc import MPC
eph = MPC.get_ephemeris(asteroid_id, step='1h', number=30 * 24)
print("type(eph) :", type(eph))
print("eph :", eph)
df_eph = eph.to_pandas()
df_eph

### 최대 움직임 속도 알아보기

In [ ]:
print(f"Proper motion: {eph['Proper motion'].quantity.to('arcmin/h').max():.03f}")

### 파일 저장하기

In [ ]:
eph = MPC.get_ephemeris(asteroid_id, location='P64',
                        ra_format={'sep': ':', 'unit': 'hourangle', 'precision': 1},
                        dec_format={'sep': ':', 'precision': 0},
                        step='1h')
print("type(eph): ", type(eph))
print("eph: ", eph)

In [ ]:
df_eph = eph.to_pandas()
df_eph['Asteroid ID'] = asteroid_id

df_eph['Asteroid name'] = df['Full name'][0]
df_eph.to_csv(str(BASEPATH/save_dir_name/ f'asteroid_{asteroid_id}.csv'))
df_eph

### (과제)

df에 있는 소행성들의 eph를 1시간 간격으로 구하여 각각 csv 파일로 저장해 보자.

In [ ]:
from pathlib import Path
from astroquery.mpc import MPC
from pprint import pprint
import re

for i, row  in df.iterrows():
    print("row['Full name'] :", row["Full name"])

    # asteroid_names = re.findall('\(([^)]+)', row['Full name'])   #extracts string in bracket()
    fullname_el = row['Full name'].split(" ")
    print("fullname_el[1] :", fullname_el[1])
    asteroid_id = fullname_el[1]
    print(asteroid_id)

    eph = MPC.get_ephemeris(asteroid_id,
                        location='P64',
                        ra_format={'sep': ':', 'unit': 'hourangle', 'precision': 1},
                        dec_format={'sep': ':', 'precision': 0},
                        step='1h')
    # print("type(eph): ", type(eph))
    #print("eph: ", eph)

    df_eph = eph.to_pandas()
    df_eph['Asteroid ID'] = asteroid_id
    df_eph['Asteroid name'] = df['Full name'][i]
    df_eph.to_csv(str(BASEPATH/save_dir_name/ f"eph_asteroid_{asteroid_id}.csv"))
